In [1]:
import pandas as pd
df = pd.read_csv('../../top_rated_wines.csv')
df = df[df['variety'].notna()] # remove any NaN values as it blows up serialization
data = df.to_dict('records') # Get only 700 records. More records will make it slower to index
len(data)

1347

In [2]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

/usr/local/python/3.12.1/lib/python3.12/site-packages/transformers/utils/generic.py:441: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [3]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

/usr/local/python/3.12.1/lib/python3.12/site-packages/transformers/utils/generic.py:309: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  _torch_pytree._register_pytree_node(


In [4]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [5]:
# Create collection to store wines
qdrant.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

/tmp/ipykernel_51018/3075695827.py:2: DeprecationWarning: `recreate_collection` method is deprecated and will be removed in the future. Use `collection_exists` to check collection existence and `create_collection` instead.
  qdrant.recreate_collection(


True

In [6]:
# vectorize!
qdrant.upload_points(
    collection_name="top_wines",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [7]:
user_prompt = "Suggest me an amazing Malbec wine from Argentina"

In [8]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'name': 'Catena Zapata Argentino Vineyard Malbec 2004', 'region': 'Argentina', 'variety': 'Red Wine', 'rating': 98.0, 'notes': '"The single-vineyard 2004 Malbec Argentino Vineyard spent 17 months in new French oak. Remarkably fragrant and complex aromatically, it offers up aromas of wood smoke, creosote, pepper, clove, black cherry, and blackberry. Made in a similar, elegant style, it is the most structured of the three single vineyard wines, needing a minimum of a decade of additional cellaring. It should easily prove to be a 25-40 year wine. It is an exceptional achievement in Malbec. When all is said and done, Catena Zapata is the Argentina winery of reference – the standard of excellence for comparing all others. The brilliant, forward-thinking Nicolas Catena remains in charge, with his daughter, Laura, playing an increasingly large role. The Catena Zapata winery is an essential destination for fans of both architecture and wine in Mendoza. It is hard to believe, given the surge i

In [9]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [11]:
import openai

openai.api_base = "http://127.0.0.1:8080/v1"
openai.api_key = "sk-no-key-required"

completion = openai.ChatCompletion.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": "Suggest me an amazing Malbec wine from Argentina"},
        {"role": "assistant", "content": str(search_results)}
    ]
)
print(completion["choices"][0]["message"]["content"])

I'd like to recommend a few amazing Malbec wines from Argentina that I think you'll love.

Firstly, I'd like to suggest the **Catena Zapata Adrianna Vineyard Malbec 2004**. This wine is a classic example of Argentine Malbec, with a rich, full-bodied texture and flavors of dark fruit, spice, and subtle oak. It's a wine that will only improve with age, and is a great representation of the Catena Zapata winery's commitment to quality and excellence.

Another great option is the **Bodega Colome Altura Maxima Malbec 2012**. This wine is a bit more modern and experimental, with a focus on showcasing the unique characteristics of the Altura Maxima Vineyard. It's a full-bodied wine with a rich, velvety texture and flavors of dark fruit, chocolate, and spice.

Lastly, I'd like to recommend the **Catena Zapata Argentino Vineyard Malbec 2004** (yes, another Catena Zapata wine!). This wine is a bit more structured and elegant than the Adrianna Vineyard, with a focus on showcasing the winery's expe